In [1]:
# import pandas as pd
# from sklearn.datasets import load_breast_cancer

# # Load the cancer dataset
# cancer = load_breast_cancer()
# df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
# df.columns = df.columns.str.replace(" ", "_")
# df["target"] = cancer.target
# df.to_parquet("cancer.parquet")
# df.head()

import duckdb
import numpy as np

with duckdb.connect("cancer.db", read_only=False) as con:
    con.execute(
        "CREATE OR REPLACE TABLE cancer AS SELECT * FROM read_parquet('cancer.parquet')"
    )

    df = con.execute("SELECT * FROM cancer").df()


df["cv"] = np.random.randint(0, 6, df.shape[0])
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,3
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,3
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,4
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,4


In [2]:
from sklearn.decomposition import PCA
import pandas as pd

pca = PCA(n_components=2)

for col in df.columns.tolist():
    if col not in ["target", "cv"]:
        df[col] = (df[col] - df[col].mean()) / df[col].std()

X = df.drop(columns=["target", "cv"])
y = df["target"]
X_pca = pca.fit_transform(X)

df_pca = pd.DataFrame(X_pca, columns=["pca1", "pca2"])
df = pd.concat([df, df_pca], axis=1)
df["is_upper_half_of_mean_radius"] = df["mean_radius"] > df["mean_radius"].median()
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv,pca1,pca2,is_upper_half_of_mean_radius
0,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,2.215566,2.253764,...,2.614365,2.107672,2.294058,2.748204,1.935312,0,3,9.184755,1.946870,True
1,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,-0.867889,...,-0.430066,-0.146620,1.086129,-0.243675,0.280943,0,3,2.385703,-3.764859,True
2,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,-0.397658,...,1.081980,0.854222,1.953282,1.151242,0.201214,0,4,5.728855,-1.074229,True
3,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,4.906602,...,3.889975,1.987839,2.173873,6.040726,4.930672,0,1,7.116691,10.266556,False
4,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,-0.009552,-0.561956,...,-0.313119,0.612640,0.728618,-0.867590,-0.396751,0,4,3.931842,-1.946359,True


In [3]:
from glam import DefaultModelData, BinomialGlmAnalysis

dat = DefaultModelData(df, y="target", cv="cv")
dat.df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv,pca1,pca2,is_upper_half_of_mean_radius
0,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,2.215566,2.253764,...,2.614365,2.107672,2.294058,2.748204,1.935312,0,3,9.184755,1.946870,True
1,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,-0.867889,...,-0.430066,-0.146620,1.086129,-0.243675,0.280943,0,3,2.385703,-3.764859,True
2,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,-0.397658,...,1.081980,0.854222,1.953282,1.151242,0.201214,0,4,5.728855,-1.074229,True
3,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,4.906602,...,3.889975,1.987839,2.173873,6.040726,4.930672,0,1,7.116691,10.266556,False
4,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,-0.009552,-0.561956,...,-0.313119,0.612640,0.728618,-0.867590,-0.396751,0,4,3.931842,-1.946359,True


In [4]:
dat

DefaultModelData(y='target', cv='cv', df.shape=(569, 35))

In [5]:
glm = BinomialGlmAnalysis(dat)
glm

BinaryGlmAnalysis(target ~ 1)

In [13]:
glm.add_feature("mean_radius")
glm.add_feature('pca1')
glm.fit()
glm

Fitting: target ~ mean_radius + pca1


BinaryGlmAnalysis(target ~ mean_radius + pca1)

In [14]:
glm.evaluate_new_feature('mean_texture')

,Deviance,DoF,p-value
Model,,,
mean_radius + pca1 + mean_texture,138.03,471,1.9e-05
mean_radius + pca1,156.32,472,


In [9]:
glm.models.model.deviance, glm2.models.model.deviance

(np.float64(156.31722981044186), np.float64(156.31722981044186))

In [10]:
glm.residual_calculator.leverage_calculator.calculate()

0      0.000035
1      0.001304
2      0.000159
3      0.017473
4      0.000446
         ...   
470    0.002156
471    0.000544
472    0.013737
473    0.000002
474    0.000030
Length: 475, dtype: float64

In [11]:
glm.aic

162.31722981044186

In [12]:
glm.leverage

0      0.000035
1      0.001304
2      0.000159
3      0.017473
4      0.000446
         ...   
470    0.002156
471    0.000544
472    0.013737
473    0.000002
474    0.000030
Length: 475, dtype: float64